# Evaluación y Validación de Clusters


## Métricas Internas, Métricas Externas y Estabilidad

---
### Objetivos del Notebook

1. Calcular e interpretar métricas internas: Silueta, Calinski-Harabasz y Davies-Bouldin
2. Visualizar el análisis de silueta por cluster para diagnóstico detallado
3. Aplicar métricas externas: ARI, NMI, V-measure y Fowlkes-Mallows
4. Evaluar la estabilidad de clusters mediante técnicas de bootstrap
5. Utilizar métricas para selección del número óptimo de clusters

---

## 1. Configuración del Entorno

In [ ]:
# Bibliotecas fundamentales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn: clustering
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture

# Scikit-learn: métricas internas
from sklearn.metrics import (
    silhouette_score,
    silhouette_samples,
    calinski_harabasz_score,
    davies_bouldin_score
)

# Scikit-learn: métricas externas
from sklearn.metrics import (
    adjusted_rand_score,
    normalized_mutual_info_score,
    homogeneity_score,
    completeness_score,
    v_measure_score,
    fowlkes_mallows_score
)

# Scikit-learn: datasets y preprocesamiento
from sklearn.datasets import make_blobs, make_moons, load_iris, load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['axes.labelsize'] = 11

# Reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Entorno configurado correctamente.")

## 2. Métricas Internas: Fundamentos

Las métricas internas evalúan la calidad del clustering utilizando únicamente los datos y las asignaciones, sin requerir etiquetas verdaderas (ground truth).

In [ ]:
# Generar datasets con diferentes estructuras de clustering
np.random.seed(RANDOM_STATE)

# Dataset 1: Clusters bien separados
X_separados, y_separados = make_blobs(n_samples=300, centers=3,
                                       cluster_std=0.5, random_state=RANDOM_STATE)

# Dataset 2: Clusters solapados
X_solapados, y_solapados = make_blobs(n_samples=300, centers=3,
                                       cluster_std=4.0, random_state=RANDOM_STATE)

# Dataset 3: Clusters de diferentes tamaños y densidades
X1 = np.random.randn(150, 2) * 0.5 + [0, 0]
X2 = np.random.randn(50, 2) * 0.3 + [4, 4]
X3 = np.random.randn(100, 2) * 1.5 + [-3, 4]
X_desbalanceado = np.vstack([X1, X2, X3])
y_desbalanceado = np.array([0]*150 + [1]*50 + [2]*100)

# Visualizar los datasets
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

datasets = [
    (X_separados, y_separados, 'Clusters bien separados'),
    (X_solapados, y_solapados, 'Clusters solapados'),
    (X_desbalanceado, y_desbalanceado, 'Clusters desbalanceados')
]

for ax, (X, y, titulo) in zip(axes, datasets):
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', edgecolors='w', s=40)
    ax.set_title(titulo)
    ax.set_xlabel('X1')
    ax.set_ylabel('X2')

plt.suptitle('Datasets de ejemplo para evaluación', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 2.1 Coeficiente de Silueta (Silhouette Score)

El coeficiente de silueta mide qué tan similar es un punto a su propio cluster comparado con otros clusters.

$$s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}$$

donde:
- $a(i)$: distancia media a puntos del mismo cluster
- $b(i)$: distancia media al cluster más cercano

In [ ]:
# Calcular silueta para cada dataset con K-Means
print("Coeficiente de Silueta (Silhouette Score)")
print("=" * 50)
print("Rango: [-1, 1], Mayor = Mejor")
print("=" * 50)

for X, y, titulo in datasets:
    kmeans = KMeans(n_clusters=3, random_state=RANDOM_STATE, n_init=10)
    labels = kmeans.fit_predict(X)

    # Silueta global
    silueta_global = silhouette_score(X, labels)

    # Silueta por punto
    siluetas_individuales = silhouette_samples(X, labels)

    print(f"\n{titulo}:")
    print(f"  Silueta global: {silueta_global:.4f}")
    print(f"  Silueta media por cluster:")
    for i in range(3):
        mask = labels == i
        print(f"    Cluster {i}: {siluetas_individuales[mask].mean():.4f} (n={mask.sum()})")

In [ ]:
def plot_silhouette_analysis(X, labels, titulo):
    """
    Genera un gráfico de análisis de silueta detallado por cluster.
    """
    n_clusters = len(np.unique(labels))
    silhouette_avg = silhouette_score(X, labels)
    sample_silhouette_values = silhouette_samples(X, labels)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Gráfico de silueta
    y_lower = 10
    colors = cm.viridis(np.linspace(0, 1, n_clusters))

    for i in range(n_clusters):
        # Obtener siluetas del cluster i y ordenarlas
        ith_cluster_silhouette_values = sample_silhouette_values[labels == i]
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=colors[i], edgecolor=colors[i], alpha=0.7)

        # Etiqueta del cluster
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i), fontsize=10)
        y_lower = y_upper + 10

    # Línea vertical para la silueta promedio
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--",
                label=f'Silueta media: {silhouette_avg:.3f}')
    ax1.axvline(x=0, color="gray", linestyle="-", alpha=0.5)

    ax1.set_xlabel('Coeficiente de silueta')
    ax1.set_ylabel('Cluster')
    ax1.set_title('Análisis de silueta por cluster')
    ax1.set_xlim([-0.3, 1])
    ax1.legend(loc='upper right')

    # Gráfico de dispersión con colores
    ax2.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis',
                edgecolors='w', s=40)
    ax2.set_xlabel('X1')
    ax2.set_ylabel('X2')
    ax2.set_title('Asignación de clusters')

    plt.suptitle(titulo, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

In [ ]:
# Análisis de silueta visual para cada dataset
for X, y, titulo in datasets:
    kmeans = KMeans(n_clusters=3, random_state=RANDOM_STATE, n_init=10)
    labels = kmeans.fit_predict(X)
    plot_silhouette_analysis(X, labels, titulo)

### 2.2 Índice de Calinski-Harabasz

También conocido como Variance Ratio Criterion, mide la relación entre la dispersión inter-cluster y la dispersión intra-cluster.

$$CH = \frac{B / (K-1)}{W / (n-K)}$$

donde B es la dispersión between-cluster y W la dispersión within-cluster.

In [ ]:
# Calcular Calinski-Harabasz para cada dataset
print("Índice de Calinski-Harabasz")
print("=" * 50)
print("Rango: [0, +inf), Mayor = Mejor")
print("=" * 50)

for X, y, titulo in datasets:
    kmeans = KMeans(n_clusters=3, random_state=RANDOM_STATE, n_init=10)
    labels = kmeans.fit_predict(X)
    ch_score = calinski_harabasz_score(X, labels)
    print(f"\n{titulo}: {ch_score:.2f}")

### 2.3 Índice de Davies-Bouldin

Mide la similitud promedio entre cada cluster y su cluster más parecido.

$$DB = \frac{1}{K} \sum_{i=1}^{K} \max_{j \neq i} R_{ij}$$

donde $R_{ij} = \frac{s_i + s_j}{d(c_i, c_j)}$

In [ ]:
# Calcular Davies-Bouldin para cada dataset
print("Índice de Davies-Bouldin")
print("=" * 50)
print("Rango: [0, +inf), Menor = Mejor")
print("=" * 50)

for X, y, titulo in datasets:
    kmeans = KMeans(n_clusters=3, random_state=RANDOM_STATE, n_init=10)
    labels = kmeans.fit_predict(X)
    db_score = davies_bouldin_score(X, labels)
    print(f"\n{titulo}: {db_score:.4f}")

### 2.4 Comparación de Métricas Internas

In [ ]:
# Comparación completa de métricas internas
resultados_internos = []

for X, y, titulo in datasets:
    kmeans = KMeans(n_clusters=3, random_state=RANDOM_STATE, n_init=10)
    labels = kmeans.fit_predict(X)

    resultados_internos.append({
        'Dataset': titulo,
        'Silueta': silhouette_score(X, labels),
        'Calinski-Harabasz': calinski_harabasz_score(X, labels),
        'Davies-Bouldin': davies_bouldin_score(X, labels)
    })

df_internos = pd.DataFrame(resultados_internos)
print("\nComparación de Métricas Internas:")
print("=" * 70)
print(df_internos.to_string(index=False))
print("\nNota: Silueta y CH son mejores cuando son mayores; DB es mejor cuando es menor.")

## 3. Métricas Externas

Las métricas externas comparan las etiquetas predichas con etiquetas verdaderas (ground truth).

In [ ]:
# Aplicar diferentes algoritmos de clustering
algoritmos = {
    'K-Means': KMeans(n_clusters=3, random_state=RANDOM_STATE, n_init=10),
    'Hierarchical (Ward)': AgglomerativeClustering(n_clusters=3, linkage='ward'),
    'GMM': GaussianMixture(n_components=3, random_state=RANDOM_STATE)
}

print("Métricas Externas - Dataset con clusters bien separados")
print("=" * 80)

### 3.1 Adjusted Rand Index (ARI)

Probamos resultados de ARI con dataset bien separado

In [ ]:
# Usar el dataset de clusters bien separados con ground truth conocido
X = X_separados.copy()
y_true = y_separados.copy()

print("Métricas Externas - Dataset con clusters bien separados")
print("=" * 80)

print("\nAdjusted Rand Index (ARI)")
print("-" * 50)
print("Rango: [-1, 1], 1 = perfecto, 0 = aleatorio")
print("-" * 50)

for nombre, modelo in algoritmos.items():
  labels = modelo.fit_predict(X)
  ari = adjusted_rand_score(y_true, labels)
  print(f"{nombre}: {ari:.4f}")

In [ ]:
# Demostración del ajuste por azar en ARI
print("\nDemostración: ARI con asignaciones aleatorias")
print("-" * 50)

aris_aleatorios = []
for i in range(100):
    labels_aleatorios = np.random.randint(0, 3, size=len(y_true))
    ari = adjusted_rand_score(y_true, labels_aleatorios)
    aris_aleatorios.append(ari)

print(f"ARI medio con etiquetas aleatorias (100 iteraciones): {np.mean(aris_aleatorios):.4f}")

print("\nEl ARI está ajustado para que asignaciones aleatorias tengan valor esperado 0.")

Comprobamos que en dataset más solapado, la métrica sale peor

In [ ]:
# Usar el dataset de clusters solapados con ground truth conocido
X = X_solapados.copy()
y_true = y_solapados.copy()

print("Métricas Externas - Dataset con clusters solapados")
print("=" * 80)

print("\nAdjusted Rand Index (ARI)")

for nombre, modelo in algoritmos.items():
  labels = modelo.fit_predict(X)
  ari = adjusted_rand_score(y_true, labels)
  print(f"{nombre}: {ari:.4f}")

### 3.2 Normalized Mutual Information (NMI)

In [ ]:
# Usar el dataset de clusters bien separados con ground truth conocido
X = X_separados.copy()
y_true = y_separados.copy()

In [ ]:
# # O Usar el dataset de clusters solapados con ground truth conocido
# X = X_solapados.copy()
# y_true = y_solapados.copy()

In [ ]:
print("\nNormalized Mutual Information (NMI)")
print("-" * 50)
print("Rango: [0, 1], 1 = correspondencia perfecta")
print("-" * 50)

for nombre, modelo in algoritmos.items():
    modelo.fit_predict(X)

    nmi = normalized_mutual_info_score(y_true, labels)
    print(f"{nombre}: {nmi:.4f}")

### 3.3 Homogeneidad, Completitud y V-measure

In [ ]:
print("\nHomogeneidad, Completitud y V-measure")
print("-" * 70)
print("Homogeneidad: cada cluster contiene solo miembros de una clase")
print("Completitud: todos los miembros de una clase están en el mismo cluster")
print("V-measure: media armónica de homogeneidad y completitud")
print("-" * 70)

for nombre, modelo in algoritmos.items():
  labels = modelo.fit_predict(X)
  h = homogeneity_score(y_true, labels)
  c = completeness_score(y_true, labels)
  v = v_measure_score(y_true, labels)

  print(f"\n{nombre}:")
  print(f"  Homogeneidad: {h:.4f}")
  print(f"  Completitud:  {c:.4f}")
  print(f"  V-measure:    {v:.4f}")

In [ ]:
# Demostración de la diferencia entre homogeneidad y completitud
print("\nDemostración: Casos extremos de homogeneidad y completitud")
print("=" * 60)

# Caso 1: Muchos clusters pequeños (alta homogeneidad, baja completitud)
labels_muchos = np.arange(len(y_true))  # Cada punto es su propio cluster
print("\nCaso: Cada punto es su propio cluster")
print(f"  Homogeneidad: {homogeneity_score(y_true, labels_muchos):.4f} (perfecta)")
print(f"  Completitud:  {completeness_score(y_true, labels_muchos):.4f} (mínima)")

# Caso 2: Un solo cluster (baja homogeneidad, alta completitud)
labels_uno = np.zeros(len(y_true), dtype=int)
print("\nCaso: Todos en un solo cluster")
print(f"  Homogeneidad: {homogeneity_score(y_true, labels_uno):.4f} (mínima)")
print(f"  Completitud:  {completeness_score(y_true, labels_uno):.4f} (perfecta)")

### 3.4 Fowlkes-Mallows Index (FMI)

In [ ]:
print("\nFowlkes-Mallows Index (FMI)")
print("-" * 50)
print("Rango: [0, 1], 1 = clustering perfecto")
print("Media geométrica de precision y recall sobre pares")
print("-" * 50)

for nombre, modelo in algoritmos.items():
  labels = modelo.fit_predict(X)
  fmi = fowlkes_mallows_score(y_true, labels)
  print(f"{nombre}: {fmi:.4f}")

### 3.5 Comparación Completa de Métricas Externas

In [ ]:
# Tabla comparativa completa
resultados_externos = []

for nombre, modelo in algoritmos.items():
    labels = modelo.fit_predict(X)

    resultados_externos.append({
        'Algoritmo': nombre,
        'ARI': adjusted_rand_score(y_true, labels),
        'NMI': normalized_mutual_info_score(y_true, labels),
        'Homog.': homogeneity_score(y_true, labels),
        'Complet.': completeness_score(y_true, labels),
        'V-measure': v_measure_score(y_true, labels),
        'FMI': fowlkes_mallows_score(y_true, labels)
    })

df_externos = pd.DataFrame(resultados_externos)
print("\nComparación completa de métricas externas:")
print("=" * 80)
print(df_externos.to_string(index=False))

In [ ]:
# O Usar el dataset de clusters solapados con ground truth conocido
X = X_solapados.copy()
y_true = y_solapados.copy()

# Tabla comparativa completa
resultados_externos = []

for nombre, modelo in algoritmos.items():
    labels = modelo.fit_predict(X)

    resultados_externos.append({
        'Algoritmo': nombre,
        'ARI': adjusted_rand_score(y_true, labels),
        'NMI': normalized_mutual_info_score(y_true, labels),
        'Homog.': homogeneity_score(y_true, labels),
        'Complet.': completeness_score(y_true, labels),
        'V-measure': v_measure_score(y_true, labels),
        'FMI': fowlkes_mallows_score(y_true, labels)
    })

df_externos = pd.DataFrame(resultados_externos)
print("\nComparación completa de métricas externas:")
print("=" * 80)
print(df_externos.to_string(index=False))

## 4. Selección del Número de Clusters

Utilizamos métricas internas para determinar el número óptimo de clusters K.

In [ ]:
# Generar dataset para selección de K
np.random.seed(RANDOM_STATE)
X_seleccion, y_seleccion = make_blobs(n_samples=500, centers=5,
                                       cluster_std=1.0, random_state=RANDOM_STATE)

# Calcular métricas para diferentes valores de K
rango_k = range(2, 11)
metricas_k = {
    'Silueta': [],
    'Calinski-Harabasz': [],
    'Davies-Bouldin': [],
    'Inercia': []
}

for k in rango_k:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    labels = kmeans.fit_predict(X_seleccion)

    metricas_k['Silueta'].append(silhouette_score(X_seleccion, labels))
    metricas_k['Calinski-Harabasz'].append(calinski_harabasz_score(X_seleccion, labels))
    metricas_k['Davies-Bouldin'].append(davies_bouldin_score(X_seleccion, labels))
    metricas_k['Inercia'].append(kmeans.inertia_)

In [ ]:
# Visualización de métricas vs K
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Silueta
ax = axes[0, 0]
ax.plot(list(rango_k), metricas_k['Silueta'], 'b-o', linewidth=2, markersize=8)
ax.axvline(x=5, color='red', linestyle='--', alpha=0.7, label='K real = 5')
k_opt_sil = list(rango_k)[np.argmax(metricas_k['Silueta'])]
ax.axvline(x=k_opt_sil, color='green', linestyle=':', alpha=0.7, label=f'K óptimo = {k_opt_sil}')
ax.set_xlabel('Número de clusters (K)')
ax.set_ylabel('Silueta')
ax.set_title('Coeficiente de Silueta (Mayor = Mejor)')
ax.legend()
ax.grid(True, alpha=0.3)

# Calinski-Harabasz
ax = axes[0, 1]
ax.plot(list(rango_k), metricas_k['Calinski-Harabasz'], 'g-o', linewidth=2, markersize=8)
ax.axvline(x=5, color='red', linestyle='--', alpha=0.7, label='K real = 5')
k_opt_ch = list(rango_k)[np.argmax(metricas_k['Calinski-Harabasz'])]
ax.axvline(x=k_opt_ch, color='green', linestyle=':', alpha=0.7, label=f'K óptimo = {k_opt_ch}')
ax.set_xlabel('Número de clusters (K)')
ax.set_ylabel('Calinski-Harabasz')
ax.set_title('Índice de Calinski-Harabasz (Mayor = Mejor)')
ax.legend()
ax.grid(True, alpha=0.3)

# Davies-Bouldin
ax = axes[1, 0]
ax.plot(list(rango_k), metricas_k['Davies-Bouldin'], 'r-o', linewidth=2, markersize=8)
ax.axvline(x=5, color='red', linestyle='--', alpha=0.7, label='K real = 5')
k_opt_db = list(rango_k)[np.argmin(metricas_k['Davies-Bouldin'])]
ax.axvline(x=k_opt_db, color='green', linestyle=':', alpha=0.7, label=f'K óptimo = {k_opt_db}')
ax.set_xlabel('Número de clusters (K)')
ax.set_ylabel('Davies-Bouldin')
ax.set_title('Índice de Davies-Bouldin (Menor = Mejor)')
ax.legend()
ax.grid(True, alpha=0.3)

# Método del codo (Inercia)
ax = axes[1, 1]
ax.plot(list(rango_k), metricas_k['Inercia'], 'm-o', linewidth=2, markersize=8)
ax.axvline(x=5, color='red', linestyle='--', alpha=0.7, label='K real = 5')
ax.set_xlabel('Número de clusters (K)')
ax.set_ylabel('Inercia (WCSS)')
ax.set_title('Método del Codo')
ax.legend()
ax.grid(True, alpha=0.3)

plt.suptitle('Selección del número óptimo de clusters', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\nK óptimo según cada métrica:")
print(f"  Silueta: {k_opt_sil}")
print(f"  Calinski-Harabasz: {k_opt_ch}")
print(f"  Davies-Bouldin: {k_opt_db}")
print(f"  K real: 5")

## 5. Estabilidad de Clusters

Evaluamos la robustez del clustering mediante técnicas de remuestreo (bootstrap).

In [ ]:
def evaluar_estabilidad_bootstrap(X, n_clusters, n_bootstrap=50, subsample_ratio=0.8):
    """
    Evalúa la estabilidad del clustering mediante bootstrap.

    Retorna:
    - Lista de ARI entre clusterings de diferentes muestras bootstrap
    """
    n_samples = len(X)
    subsample_size = int(n_samples * subsample_ratio)

    # Almacenar resultados de cada iteración
    all_labels = []
    all_indices = []

    for i in range(n_bootstrap):
        # Submuestra aleatoria
        indices = np.random.choice(n_samples, size=subsample_size, replace=False)
        X_sub = X[indices]

        # Clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=i, n_init=10)
        labels = kmeans.fit_predict(X_sub)

        all_labels.append(labels)
        all_indices.append(indices)

    # Calcular ARI entre pares de particiones (en puntos comunes)
    aris = []
    for i in range(n_bootstrap):
        for j in range(i+1, n_bootstrap):
            # Encontrar puntos en común
            common = np.intersect1d(all_indices[i], all_indices[j])
            if len(common) > 10:
                # Obtener etiquetas para puntos comunes
                mask_i = np.isin(all_indices[i], common)
                mask_j = np.isin(all_indices[j], common)

                labels_i = all_labels[i][mask_i]
                labels_j = all_labels[j][mask_j]

                # Reordenar para que coincidan los índices
                order_i = np.argsort(all_indices[i][mask_i])
                order_j = np.argsort(all_indices[j][mask_j])

                ari = adjusted_rand_score(labels_i[order_i], labels_j[order_j])
                aris.append(ari)

    return aris

In [ ]:
# Evaluar estabilidad para diferentes valores de K
print("Análisis de estabilidad mediante bootstrap")
print("=" * 60)

# Usar el dataset de 5 clusters
estabilidades = {}

for k in [3, 4, 5, 6, 7]:
    aris = evaluar_estabilidad_bootstrap(X_seleccion, n_clusters=k, n_bootstrap=30)
    estabilidades[k] = aris
    print(f"K={k}: Estabilidad media = {np.mean(aris):.4f} (+/- {np.std(aris):.4f})")

In [ ]:
# Visualización de estabilidad
fig, ax = plt.subplots(figsize=(12, 6))

ks = list(estabilidades.keys())
means = [np.mean(estabilidades[k]) for k in ks]
stds = [np.std(estabilidades[k]) for k in ks]

ax.errorbar(ks, means, yerr=stds, fmt='o-', linewidth=2, markersize=10,
            capsize=5, capthick=2)
ax.axvline(x=5, color='red', linestyle='--', alpha=0.7, label='K real = 5')
ax.axhline(y=1.0, color='gray', linestyle=':', alpha=0.5)

ax.set_xlabel('Número de clusters (K)')
ax.set_ylabel('Estabilidad (ARI medio entre bootstrap)')
ax.set_title('Estabilidad del clustering vs número de clusters')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim([0, 1.1])

plt.tight_layout()
plt.show()

print("\nInterpretación:")
print("- Alta estabilidad (~1.0) indica que el clustering es robusto")
print("- El K óptimo suele tener la mayor estabilidad")
print("- Varianza alta indica sensibilidad a los datos de entrada")

## 6. Caso Práctico: Dataset Wine

Aplicamos todas las métricas a un dataset real multidimensional.

In [ ]:
# Cargar y preparar dataset Wine
wine = load_wine()
X_wine = wine.data
y_wine = wine.target

# Estandarizar
scaler = StandardScaler()
X_wine_scaled = scaler.fit_transform(X_wine)

# Reducir a 2D para visualización
pca = PCA(n_components=2)
X_wine_2d = pca.fit_transform(X_wine_scaled)

print(f"Dataset Wine:")
print(f"  Muestras: {X_wine.shape[0]}")
print(f"  Características: {X_wine.shape[1]}")
print(f"  Clases reales: {len(np.unique(y_wine))}")
print(f"  Varianza explicada por 2 PCs: {pca.explained_variance_ratio_.sum():.2%}")

In [ ]:
# Comparación completa de algoritmos
algoritmos_wine = {
    'K-Means': KMeans(n_clusters=3, random_state=RANDOM_STATE, n_init=10),
    'Hierarchical': AgglomerativeClustering(n_clusters=3, linkage='ward'),
    'GMM': GaussianMixture(n_components=3, random_state=RANDOM_STATE),
}

resultados_wine = []

for nombre, modelo in algoritmos_wine.items():
    labels = modelo.fit_predict(X_wine_scaled)

    resultado = {
        'Algoritmo': nombre,
        # Métricas internas
        'Silueta': silhouette_score(X_wine_scaled, labels),
        'CH': calinski_harabasz_score(X_wine_scaled, labels),
        'DB': davies_bouldin_score(X_wine_scaled, labels),
        # Métricas externas
        'ARI': adjusted_rand_score(y_wine, labels),
        'NMI': normalized_mutual_info_score(y_wine, labels),
        'V-measure': v_measure_score(y_wine, labels),
        'FMI': fowlkes_mallows_score(y_wine, labels)
    }
    resultados_wine.append(resultado)

df_wine = pd.DataFrame(resultados_wine)
print("\nResultados en dataset Wine:")
print("=" * 100)
print(df_wine.to_string(index=False))

In [ ]:
# Visualización comparativa
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Ground truth
ax = axes[0, 0]
ax.scatter(X_wine_2d[:, 0], X_wine_2d[:, 1], c=y_wine, cmap='viridis',
           edgecolors='w', s=50)
ax.set_title('Ground Truth')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

# K-Means
ax = axes[0, 1]
labels_km = KMeans(n_clusters=3, random_state=RANDOM_STATE, n_init=10).fit_predict(X_wine_scaled)
ax.scatter(X_wine_2d[:, 0], X_wine_2d[:, 1], c=labels_km, cmap='viridis',
           edgecolors='w', s=50)
ari_km = adjusted_rand_score(y_wine, labels_km)
sil_km = silhouette_score(X_wine_scaled, labels_km)
ax.set_title(f'K-Means\nARI: {ari_km:.3f}, Silueta: {sil_km:.3f}')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

# Hierarchical
ax = axes[1, 0]
labels_hc = AgglomerativeClustering(n_clusters=3, linkage='ward').fit_predict(X_wine_scaled)
ax.scatter(X_wine_2d[:, 0], X_wine_2d[:, 1], c=labels_hc, cmap='viridis',
           edgecolors='w', s=50)
ari_hc = adjusted_rand_score(y_wine, labels_hc)
sil_hc = silhouette_score(X_wine_scaled, labels_hc)
ax.set_title(f'Hierarchical (Ward)\nARI: {ari_hc:.3f}, Silueta: {sil_hc:.3f}')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

# GMM
ax = axes[1, 1]
labels_gmm = GaussianMixture(n_components=3, random_state=RANDOM_STATE).fit_predict(X_wine_scaled)
ax.scatter(X_wine_2d[:, 0], X_wine_2d[:, 1], c=labels_gmm, cmap='viridis',
           edgecolors='w', s=50)
ari_gmm = adjusted_rand_score(y_wine, labels_gmm)
sil_gmm = silhouette_score(X_wine_scaled, labels_gmm)
ax.set_title(f'GMM\nARI: {ari_gmm:.3f}, Silueta: {sil_gmm:.3f}')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

plt.suptitle('Comparación de algoritmos en dataset Wine', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Análisis de silueta detallado para K-Means en Wine
plot_silhouette_analysis(X_wine_scaled, labels_km, 'Análisis de silueta - K-Means en Wine')

## 7. Métricas en Escenarios Problemáticos

Evaluamos cómo se comportan las métricas cuando el clustering falla.

In [ ]:
# Dataset con estructura no convexa (moons)
X_moons, y_moons = make_moons(n_samples=300, noise=0.1, random_state=RANDOM_STATE)

# Aplicar K-Means (que fallará) vs el clustering real
kmeans_moons = KMeans(n_clusters=2, random_state=RANDOM_STATE, n_init=10)
labels_kmeans_moons = kmeans_moons.fit_predict(X_moons)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Ground truth
ax = axes[0]
ax.scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='viridis',
           edgecolors='w', s=50)
ax.set_title('Ground Truth')
ax.set_xlabel('X1')
ax.set_ylabel('X2')

# K-Means
ax = axes[1]
ax.scatter(X_moons[:, 0], X_moons[:, 1], c=labels_kmeans_moons, cmap='viridis',
           edgecolors='w', s=50)
ax.scatter(kmeans_moons.cluster_centers_[:, 0], kmeans_moons.cluster_centers_[:, 1],
           c='red', marker='X', s=200, edgecolors='w')
ax.set_title('K-Means (clustering incorrecto)')
ax.set_xlabel('X1')
ax.set_ylabel('X2')

plt.suptitle('K-Means en datos no convexos (moons)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Calcular métricas
print("\nMétricas para K-Means en dataset moons:")
print("=" * 50)
print("\nMétricas internas (sin ground truth):")
print(f"  Silueta: {silhouette_score(X_moons, labels_kmeans_moons):.4f}")
print(f"  Calinski-Harabasz: {calinski_harabasz_score(X_moons, labels_kmeans_moons):.2f}")
print(f"  Davies-Bouldin: {davies_bouldin_score(X_moons, labels_kmeans_moons):.4f}")

print("\nMétricas externas (con ground truth):")
print(f"  ARI: {adjusted_rand_score(y_moons, labels_kmeans_moons):.4f}")
print(f"  NMI: {normalized_mutual_info_score(y_moons, labels_kmeans_moons):.4f}")

print("\nNota: Las métricas internas pueden dar valores 'buenos' incluso cuando")
print("el clustering es incorrecto, porque asumen clusters convexos.")

## 8. Resumen y Conclusiones

### Guía de Selección de Métricas

**Métricas Internas (sin ground truth):**

| Métrica | Rango | Óptimo | Uso Principal | Limitación |
|---------|-------|--------|---------------|------------|
| Silueta | [-1, 1] | Mayor | Diagnóstico por punto | O(n²) |
| Calinski-Harabasz | [0, +∞) | Mayor | Selección de K rápida | Asume convexidad |
| Davies-Bouldin | [0, +∞) | Menor | Detectar clusters similares | Solo usa centroides |

**Métricas Externas (con ground truth):**

| Métrica | Rango | Ajustada | Uso Principal |
|---------|-------|----------|---------------|
| ARI | [-1, 1] | Sí | Benchmarking general |
| NMI | [0, 1] | No (AMI sí) | Comparar particiones |
| V-measure | [0, 1] | No | Diagnóstico h/c |
| FMI | [0, 1] | No | Balance precision/recall |

### Recomendaciones Prácticas

1. **Usar múltiples métricas**: ninguna métrica es perfecta
2. **Preferir métricas ajustadas** (ARI, AMI) para comparaciones justas
3. **Complementar con visualización** siempre que sea posible
4. **Evaluar estabilidad** mediante bootstrap para clusterings robustos
5. **Considerar el contexto del problema**: la interpretabilidad a veces importa más que las métricas

---

## Referencias

- Rousseeuw, P. J. (1987). Silhouettes: a graphical aid to the interpretation and validation of cluster analysis. Journal of Computational and Applied Mathematics, 20, 53-65.
- Calinski, T., & Harabasz, J. (1974). A dendrite method for cluster analysis. Communications in Statistics, 3(1), 1-27.
- Davies, D. L., & Bouldin, D. W. (1979). A cluster separation measure. IEEE TPAMI, 1(2), 224-227.
- Hubert, L., & Arabie, P. (1985). Comparing partitions. Journal of Classification, 2(1), 193-218.
- Vinh, N. X., Epps, J., & Bailey, J. (2010). Information theoretic measures for clusterings comparison. ICML.
- Scikit-learn documentation: https://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation

---


# EOF (End Of File)